# Hartree-Fock 

#### Introduction
Hartree fock method is the simplest physically relevant method of solving the time-independent Schrodinger equation for a many-body system.

The genius of this method lies in the gradual solving of the problem for individual electrons. Problematic electron-electron interaction is dealth with by considering the electron as existing in an "averaged" potential of other electrons.

#### Initial approximations
Hartree Fock method makes five major simplifications/assumptions to deal with the Schrodingers equation:

- **Born-Oppenheimer approximation**: separation of nuclear and electronic movement
- **Model of independent electrons**
- **Wavefunction in a Slater determinant form**
- **MO LCAO**: Molecular orbitals as Linear Combination of Atomic Orbitals

____

#### SCF procedure
**0. Geometry and basis set**

- **Geometry**: The positions of the nuclei determine the potential field in which the electrons move. 

- **Basis Set**: Basis set consists of a predefined set of functions $\phi$ (atomic orbitals - AO) centered on atom nuclei that are used to construct molecular orbitals (MO). These functions typically Gaussian functions, Slater-type orbitals or exact solutions from the hydrogen atom. They serve as the building block for approximating the wavefunction of the system.

**1. Starting MO - set of coefficients $c_{iA}$**

Molecular orbitals are constructed as linear combinations of atomic orbitals using MO LCAO:

$\psi_i (r) = \sum_A c_{iA} \Psi_A (r)$

where $\psi_i(r)$ is the $i$-th molecular orbital, $c_{iA}$ are coefficients weighing the contribution of each basis function (atomic orbital) $\phi_A(r)$ to the total molecular orbital.

An initial guess for the molecular orbital coefficents $c_{iA}$ provides a starting point for the iterative SCF process. This guess can be based on simpler methods like Hucker theory. These estimated coefficients are then refined throughout the SCF process thanks to the *Variational theorem* which states:

$E_0 \le \langle \psi \hat{H} \psi \rangle$

that means that the eigenvalue (energy) of any trial wavefunction is greater or equal than the true ground state energy. The implications of this are massive and allow for the iterative minimization of energy with respect to the coefficients $c_{iA}$ to yield the lowest possible energy within the chosen basis set.

**2. Calculating the integrals $\langle AB|CD \rangle$, $S_{AB}$, $h_{AB}$**
- Overlap integrals $S_{AB}$ measure the overlap between basis functions. Basis sets are generally not orthonormal. And in order to write the Fock Equations equations in the form of an matrix eigenvalue problem, we need to orthogonalize our basis, hence why including the $S$ matrix. 

    $S_{AB} = \langle \phi_A | \phi_B \rangle = \int \phi_A \phi_B dr$

- Core Hamiltonian $h_{AB}$: Represents the kinetic energy of electrons and their attraction to the nucleic. It forms the starting point for building the Fock matrix.

    $h_{AB} = \int \phi_A (r) \bigg( - \frac{1}{2} \nabla^2 - \sum_N \frac{Z_N}{|r-R_N|}\bigg) \phi_B (r) dr$

- Two electron repulsion integrals $\langle AB|CD \rangle$ account for electron electron repulsion. These integrals are crucial for incorporating electron correlation effects within the mean-field approximation

**3. Create matrix P_{AB}**

Density matrix encodes the distribution of electrons over the molecular orbitals. It is used to calculate the electron density and thus the effective potential experienced by each electron.

**4. Create matrix F_{AB}**

**5. Solving Fock equations**

**6. Checking convergence criteria - back to item 3**

____

In [91]:
import sys
import math
import numpy as np
import json
import os

class BasisFunction:
    def __init__(self, atom_number, element, unique_id, coordinates, angular_momentum, exponents, coefficients):
        self.atom_number = atom_number
        self.element = element
        self.unique_id = unique_id
        self.coordinates = coordinates
        self.angular_momentum = angular_momentum
        self.exponents = exponents
        self.coefficients = coefficients

    def __repr__(self):
        return f"Shell(atom_number:{self.atom_number}, element:{self.element},unique_id:{self.unique_id}, coordinates:{self.coordinates}, angular_momentum:{self.angular_momentum}, exponents:{self.exponents}, coefficients:{self.coefficients})"

def load_basis_set(atomic_number, element, unique_id, coordinates, basis_set_file):
    '''
    Load basis set from json file and creates a list of BasisFunction instances for a given atomic number
    Args:
        atomic_number (int): Atomic number of element
        basis_set_file (str): Path to basis set json file

    Returns:
       electron_shell (list): List of BasisFunction instances with exponents and coefficients for a given electron shell of a given atomic number

    Raises:
        FileNotFoundError: If the file does not exist
        ValueError: If the file content is not as expected or atomic number is not found
    '''
    if os.path.exists(basis_set_file):
        with open(basis_set_file, 'r') as f:
            basis_set = json.load(f)
    else:
        raise FileNotFoundError(f"File not found: {basis_set_file}")
    
    if 'elements' in basis_set:
        if str(atomic_number) in basis_set['elements']:
            element_data = basis_set['elements'][str(atomic_number)]
            electron_shell = []
            for shell in element_data.get('electron_shells', []):
                angular_momenta_array = [int(l) for l in shell.get('angular_momentum', [])]
                angular_momentum = angular_momenta_array[len(angular_momenta_array) - 1]
                exponents = [float(e) for e in shell.get('exponents', [])]
                coefficients = [[float(c) for c in coef] for coef in shell.get('coefficients', [])]
                electron_shell.append(BasisFunction(atomic_number, element, unique_id, coordinates, angular_momentum, exponents, coefficients))
            return electron_shell
        else:
            raise ValueError(f"Atomic number {atomic_number} not found in the basis set file.")
    else:
        raise ValueError("Invalid basis set file format.")

def load_basis_set_for_molecule(atomic_numbers, elements, unique_ids, coordinate_array, basis_set_file):
    '''
    Load basis set from json file and creates a dictionary of atomic number to list of BasisFunction instances
    Args:
        atomic_numbers (list): List of atomic numbers of elements in the molecule
        basis_set_file (str): Path to basis set json file

    Returns:
        basis_set_for_molecule (dict): Dictionary with atomic numbers as keys and lists of BasisFunction instances as values
    '''
    #TODO: Error handling
    
    basis_set_for_molecule = []
    for unique_id in unique_ids:
        atomic_number = atomic_numbers[unique_id]
        basis_set_for_molecule.append(load_basis_set(atomic_number, elements[unique_id], unique_id, coordinate_array[unique_id], basis_set_file))
    
    return basis_set_for_molecule

def read_xyz(file_path):
    atomic_numbers = {
        'H': 1,
        'He': 2,
        'Li': 3,
        'Be': 4,
        'B': 5,
        'C': 6,
        'N': 7,
        'O': 8,
        'F': 9,
        'Ne': 10
        # Add more elements as needed
    }
    atoms = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        atom_count = int(lines[0].strip())
        unique_id = 0
        for line in lines[2:2+atom_count]:
            parts = line.split()
            element = parts[0]
            coordinates = [float(x) for x in parts[1:4]]
            atomic_number = atomic_numbers.get(element, None)
            if atomic_number is None:
                raise ValueError(f"Element {element} not recognized.")
            atoms.append((atomic_number, element, unique_id, coordinates))
            unique_id += 1
    return atoms

#### Overlap Matrix

In [92]:
def calculate_overlap_exponent(p1, p2, c1, c2, r1, r2):
    N = c1 * c2
    p = p1 + p2
    q = p1 * p2 / p
    Q = np.array(r1) - np.array(r2)
    Q2 = np.dot(Q, Q)
    return N * math.exp(-q * Q2) * (math.pi / p)**(3/2)

def overlap_matrix(basis_set):

    n_basis = sum([len(atom) for atom in basis_set])

    S_matrix = np.zeros([n_basis, n_basis])
    
    basis_functions = []
    for atom in basis_set:
        for shell in atom:
            basis_functions.append(shell)
    
    for i in range(n_basis):
        for j in range(n_basis):
            shell_i = basis_functions[i]
            shell_j = basis_functions[j]

            n_primitives_i = len(shell_i.exponents)
            n_primitives_j = len(shell_j.exponents)

            for k in range(n_primitives_i):
                for l in range(n_primitives_j):
                    exp_i = shell_i.exponents[k]
                    exp_j = shell_j.exponents[l]
                    for coeff_i in shell_i.coefficients:
                        for coeff_j in shell_j.coefficients:
                            for coeff_ik in coeff_i:
                                for coeff_jl in coeff_j:
                                    S_matrix[i,j] += calculate_overlap_exponent(exp_i, exp_j, coeff_ik, coeff_jl, shell_i.coordinates, shell_j.coordinates)
    
    return S_matrix


#### Kinetic Energy Integral
Now let's start with calculating the constituents of the Fock matrix, starting with the kinetic energy term $T$.

$F = T + V_{ne} + V_{ee}$

The kinetic energy integral for Gaussian functions can be calculated using a well defined formula.

$T_{ij} = \sum^{n_i}_{k=1} \sum^{n_j}_{l=1} c_{ik} c_{jl} \bigg( \frac{\pi}{p} \bigg)^{3/2} \bigg( \frac{2ab}{p} \bigg) \bigg[ 3 - 2 ab \frac{R^2}{p} \bigg] e^{- \frac{abR^2}{p}}$

where
- $a$ and $b$ are the exponents of the Gaussian functions
- $c_{ik}$ and $c_{jl}$ are the coefficients
- $p = a + b$
- $R$ is the distance between the centers of the Gaussian functions

In [93]:
def calculate_kinetic_exponent(p1, p2, c1, c2, r1, r2):
    p = p1 + p2
    q = p1 * p2 / p
    R = np.array(r1) - np.array(r2)
    R2 = np.dot(R, R)
    
    pre_factor = c1 * c2 * (math.pi / p)**(3/2) * (2 * p1 * p2 / p)
    term1 = 3
    term2 = -2 * q * R2 / p
    
    return pre_factor * (term1 + term2) * math.exp(-q * R2)

def kinetic_matrix(basis_set):
    n_basis = sum([len(atom) for atom in basis_set])

    T_matrix = np.zeros([n_basis, n_basis])
    
    basis_functions = []
    for atom in basis_set:
        for shell in atom:
            basis_functions.append(shell)
    
    for i in range(n_basis):
        for j in range(n_basis):
            shell_i = basis_functions[i]
            shell_j = basis_functions[j]

            n_primitives_i = len(shell_i.exponents)
            n_primitives_j = len(shell_j.exponents)

            for k in range(n_primitives_i):
                for l in range(n_primitives_j):
                    exp_i = shell_i.exponents[k]
                    exp_j = shell_j.exponents[l]
                    for coeff_i in shell_i.coefficients:
                        for coeff_j in shell_j.coefficients:
                            for coeff_ik in coeff_i:
                                for coeff_jl in coeff_j:
                                    T_matrix[i, j] += calculate_kinetic_exponent(exp_i, exp_j, coeff_ik, coeff_jl, shell_i.coordinates, shell_j.coordinates)
    
    return T_matrix

#### Nuclear - Electron Attraction Integral

Nuclear-electron attraction integral for Gaussian functions involves the Coulomb potential between the electron and the nuclei. This term is often computed using Boys function to handle the integral analytically.

The integral is given by:
$V_{ij} = - \sum_A Z_A \sum^{n_i}_{k=1} \sum^{n_j}_{l=1} c_{ik} c_{jl} \bigg( \frac{2 \pi}{p} \bigg) e^{- \frac{qR^2}{p}} F_0 \bigg( \frac{p R_{iA}R_{jA}}{p} \bigg)$
where:
- $Z_A$ is the nuclear charge of atom $A$
- $p=a+b$
- $q=\frac{ab}{p}$
- $R_{ij}$ is the distance between the Gaussian function centers
- $R_{iA}$ and $R_{jA}$ are the distances between Gaussian function centers and the nucleus
- $F_0$ is the Boys function

In [94]:
from scipy.special import erf

def boys_function(T):
    if T < 1e-6:
        return 1 - T / 3
    else:
        return 0.5 * math.sqrt(math.pi / T) * erf(math.sqrt(T))

def calculate_nuclear_attraction_exponent(p1, p2, c1, c2, r1, r2, rA, ZA):
    p = p1 + p2
    q = p1 * p2 / p
    RP = (p1 * np.array(r1) + p2 * np.array(r2)) / p
    RPA = np.array(RP) - np.array(rA)
    RPA2 = np.dot(RPA, RPA)
    R12 = np.array(r1) - np.array(r2)
    R12_2 = np.dot(R12, R12)

    pre_factor = c1 * c2 * (2 * math.pi / p) * math.exp(-q * R12_2)
    return -ZA * pre_factor * boys_function(p * RPA2)

def nuclear_electron_attraction_matrix(basis_set):
    n_basis = sum([len(atom) for atom in basis_set])

    VE_matrix = np.zeros([n_basis, n_basis])
    
    basis_functions = []
    nuclei_coords = []
    nuclei_charges = []

    for atom in basis_set:
        for shell in atom:
            basis_functions.append(shell)
        if atom:
            nuclei_coords.append(atom[0].coordinates)
            nuclei_charges.append(atom[0].atom_number)
    
    for i in range(n_basis):
        for j in range(n_basis):
            shell_i = basis_functions[i]
            shell_j = basis_functions[j]

            n_primitives_i = len(shell_i.exponents)
            n_primitives_j = len(shell_j.exponents)

            for k in range(n_primitives_i):
                for l in range(n_primitives_j):
                    exp_i = shell_i.exponents[k]
                    exp_j = shell_j.exponents[l]
                    for coeff_i in shell_i.coefficients:
                        for coeff_j in shell_j.coefficients:
                            for coeff_ik in coeff_i:
                                for coeff_jl in coeff_j:
                                    for rA, ZA in zip(nuclei_coords, nuclei_charges):
                                        VE_matrix[i, j] += calculate_nuclear_attraction_exponent(exp_i, exp_j, coeff_ik, coeff_jl, shell_i.coordinates, shell_j.coordinates, rA, ZA)
    
    return VE_matrix

#### Electron-electron repulsion integrals

$\langle ij \vert kl \rangle = \int \int \frac{\phi_i (r_1) \phi_j(r_1) \phi_k(r_2) \phi_l(r_2)}{\vert r_1 - r_2 \vert} dr_1 dr_2$ 

where:
- $\phi_i, \phi_j, \phi_k, \phi_l$ are Gaussian basis functions
- $r_1$ and $r_2$ are the coordinates of the two electrons

The Boys function is used again to handle the integrals analytically.

In [95]:
#def boys_function(T):
#    if T < 1e-6:
#        return 1 - T / 3
#    else:
#        return 0.5 * math.sqrt(math.pi / T) * erf(math.sqrt(T))

def calculate_two_electron_integral(p1, p2, p3, p4, c1, c2, c3, c4, r1, r2, r3, r4):
    p = p1 + p2
    q = p3 + p4
    alpha = p1 * p2 / p
    beta = p3 * p4 / q
    P = (p1 * np.array(r1) + p2 * np.array(r2)) / p
    Q = (p3 * np.array(r3) + p4 * np.array(r4)) / q
    PQ = np.array(P) - np.array(Q)
    PQ2 = np.dot(PQ, PQ)
    
    pre_factor = (2 * math.pi**(5/2)) / (p * q * math.sqrt(p + q))
    boys_arg = alpha * beta * PQ2 / (alpha + beta)
    
    return c1 * c2 * c3 * c4 * pre_factor * boys_function(boys_arg)

def electron_electron_repulsion_matrix(basis_set):
    n_basis = sum([len(atom) for atom in basis_set])
    EE_matrix = np.zeros((n_basis, n_basis, n_basis, n_basis))
    
    basis_functions = []
    for atom in basis_set:
        for shell in atom:
            basis_functions.append(shell)
    
    for i in range(n_basis):
        for j in range(n_basis):
            for k in range(n_basis):
                for l in range(n_basis):
                    shell_i = basis_functions[i]
                    shell_j = basis_functions[j]
                    shell_k = basis_functions[k]
                    shell_l = basis_functions[l]
                    
                    n_primitives_i = len(shell_i.exponents)
                    n_primitives_j = len(shell_j.exponents)
                    n_primitives_k = len(shell_k.exponents)
                    n_primitives_l = len(shell_l.exponents)
                    
                    for a in range(n_primitives_i):
                        for b in range(n_primitives_j):
                            for c in range(n_primitives_k):
                                for d in range(n_primitives_l):
                                    exp_i = shell_i.exponents[a]
                                    exp_j = shell_j.exponents[b]
                                    exp_k = shell_k.exponents[c]
                                    exp_l = shell_l.exponents[d]
                                    for coeff_i in shell_i.coefficients:
                                        for coeff_j in shell_j.coefficients:
                                            for coeff_k in shell_k.coefficients:
                                                for coeff_l in shell_l.coefficients:
                                                    for coeff_ia in coeff_i:
                                                        for coeff_jb in coeff_j:
                                                            for coeff_kc in coeff_k:
                                                                for coeff_ld in coeff_l:
                                                                    EE_matrix[i, j, k, l] += calculate_two_electron_integral(
                                                                        exp_i, exp_j, exp_k, exp_l, 
                                                                        coeff_ia, coeff_jb, coeff_kc, coeff_ld, 
                                                                        shell_i.coordinates, shell_j.coordinates, 
                                                                        shell_k.coordinates, shell_l.coordinates
                                                                    )
    return EE_matrix

#### Density matrix and initial guess

In [96]:
import scipy.linalg

def compute_initial_density_matrix(S_matrix, num_electrons):
    eigvals, eigvecs = np.linalg.eigh(S_matrix)
    S_half_inv = eigvecs @ np.diag(eigvals**-0.5) @ eigvecs.T
    
    F_matrix_guess = T_matrix + VE_matrix  # Simplified guess
    
    F_prime = S_half_inv @ F_matrix_guess @ S_half_inv.T
    
    eigvals_F, eigvecs_F = np.linalg.eigh(F_prime)
    
    C_matrix = S_half_inv @ eigvecs_F
    
    num_occ_orbitals = num_electrons // 2
    P_matrix = np.zeros_like(S_matrix)
    for i in range(num_occ_orbitals):
        C_col = C_matrix[:, i]
        P_matrix += 2 * np.outer(C_col, C_col)
    
    return P_matrix

#### Constructing the Fock matrix

In [97]:
def construct_fock_matrix(T_matrix, VE_matrix, EE_matrix, P_matrix):
    n_basis = T_matrix.shape[0]
    F_matrix = np.zeros((n_basis, n_basis))
    
    for mu in range(n_basis):
        for nu in range(n_basis):
            F_matrix[mu, nu] = T_matrix[mu, nu] + VE_matrix[mu, nu]
            for lambda_ in range(n_basis):
                for sigma in range(n_basis):
                    F_matrix[mu, nu] += P_matrix[lambda_, sigma] * (EE_matrix[mu, nu, lambda_, sigma] - 0.5 * EE_matrix[mu, lambda_, nu, sigma])
    
    return F_matrix

#### Self-Consistent Field procedure

In [98]:
def scf_procedure(S_matrix, T_matrix, VE_matrix, EE_matrix, num_electrons, max_iter=100, convergence_thresh=1e-6):
    P_matrix = compute_initial_density_matrix(S_matrix, num_electrons)
    
    for iteration in range(max_iter):
        F_matrix = construct_fock_matrix(T_matrix, VE_matrix, EE_matrix, P_matrix)
        
        eigvals, eigvecs = np.linalg.eigh(F_matrix)
        
        new_P_matrix = np.zeros_like(S_matrix)
        num_occ_orbitals = num_electrons // 2
        for i in range(num_occ_orbitals):
            C_col = eigvecs[:, i]
            new_P_matrix += 2 * np.outer(C_col, C_col)
        
        delta_P = np.linalg.norm(new_P_matrix - P_matrix)
        print(f"Iteration {iteration + 1}, Delta P: {delta_P}")
        
        if delta_P < convergence_thresh:
            print("SCF converged.")
            break
        
        P_matrix = new_P_matrix
    else:
        print("SCF did not converge within the maximum number of iterations.")
    
    return F_matrix, P_matrix

#### Total Electronic Energy

In [99]:
def compute_core_hamiltonian(T_matrix, VE_matrix):
    return T_matrix + VE_matrix

def compute_total_energy(P_matrix, H_matrix, F_matrix):
    E_total = 0.0
    n_basis = P_matrix.shape[0]
    
    for mu in range(n_basis):
        for nu in range(n_basis):
            E_total += P_matrix[mu, nu] * (H_matrix[mu, nu] + F_matrix[mu, nu])
    
    return E_total

In [100]:
molecule = read_xyz('hydrogen.xyz')

atomic_numbers = [molecule[i][0] for i in range(len(molecule))]
elements = [molecule[j][1] for j in range(len(molecule))]
unique_ids = [molecule[k][2] for k in range(len(molecule))]
coordinate_array = [molecule[l][3] for l in range(len(molecule))]

basis_set = load_basis_set_for_molecule(atomic_numbers, elements, unique_ids, coordinate_array, 'sto-3g_h_o.json')

print(basis_set)

number_of_electrons = sum([molecule[i][0] for i in range(len(molecule))])
print(number_of_electrons)

S_matrix = overlap_matrix(basis_set)
#print(S_matrix)
T_matrix = kinetic_matrix(basis_set)
#print(T_matrix)
VE_matrix = nuclear_electron_attraction_matrix(basis_set)
#print(VE_matrix)
EE_matrix = electron_electron_repulsion_matrix(basis_set)
#print(EE_matrix)

P_matrix = compute_initial_density_matrix(S_matrix, number_of_electrons)
#print(P_matrix)

F_matrix = construct_fock_matrix(T_matrix, VE_matrix, EE_matrix, P_matrix)
#print(F_matrix)

# Perform the SCF procedure
F_matrix_final, P_matrix_final = scf_procedure(S_matrix, T_matrix, VE_matrix, EE_matrix, number_of_electrons)

print("Final Fock Matrix:")
print(F_matrix_final)
print("Final Density Matrix:")
print(P_matrix_final)

H_matrix = compute_core_hamiltonian(T_matrix, VE_matrix)
total_energy = compute_total_energy(P_matrix_final, H_matrix, F_matrix_final)

print("Total Electronic Energy:", total_energy,"a.u.")

[[Shell(atom_number:1, element:H,unique_id:0, coordinates:[0.0, 0.0, -0.37], angular_momentum:0, exponents:[3.425250914, 0.6239137298, 0.168855404], coefficients:[[0.1543289673, 0.5353281423, 0.4446345422]])], [Shell(atom_number:1, element:H,unique_id:1, coordinates:[0.0, 0.0, 0.37], angular_momentum:0, exponents:[3.425250914, 0.6239137298, 0.168855404], coefficients:[[0.1543289673, 0.5353281423, 0.4446345422]])]]
2
Iteration 1, Delta P: 1.9843343812087209
Iteration 2, Delta P: 2.82842712474619
Iteration 3, Delta P: 2.8284271247461894
Iteration 4, Delta P: 2.8284271247461894
Iteration 5, Delta P: 2.8284271247461894
Iteration 6, Delta P: 2.8284271247461894
Iteration 7, Delta P: 2.8284271247461894
Iteration 8, Delta P: 2.8284271247461894
Iteration 9, Delta P: 2.8284271247461894
Iteration 10, Delta P: 2.8284271247461894
Iteration 11, Delta P: 2.8284271247461894
Iteration 12, Delta P: 2.8284271247461894
Iteration 13, Delta P: 2.8284271247461894
Iteration 14, Delta P: 2.8284271247461894
Ite